# Ex05 Introducción a TensorFlow-GPU

Hasta ahora, hemos utilizado numpy para construir redes neuronales. Ahora comenzaremos a utilizar TensorFlow como framework para construir redes neuronales profundas de una forma más rápida. Los frameworks como TensorFlow, Caffe y Keras pueden acelerar significativamente el desarrollo de modelos de aprendizaje automático. En esta actividad, aprenderemos realizar los siguiente en TensorFlow:

- Uso de elementos básicos (constantes, variables, sesiones, placeholders)
- Algoritmos de entrenamiento
- Implementar una red neuronal

$\textbf{Nota}$: los frameworks puede reducir el tiempo de codificación, pero también pueden tener optimizaciones que mejoren la velocidad de ejecución del código.

$\textbf{Observación}$: esta actividad ha sido diseñada para TensorFlow r2.0 para GPU, si utliza otra versión podría encontrar algunos detalles de sintaxis o en el uso de funciones.

## 1. Elementos básicos de TensorFlow

### 1.1 Uso de liberías

Comencemos por importar la librería `tensorflow` como `tf`. En caso de que aun no tenga instalado TensorFlow, se puede optar por las siguientes opciones:

- [Utilizar un docker](https://www.tensorflow.org/install#run-a-tensorflow-container).
- [Utilizar Google colab](https://www.tensorflow.org/install#google-colab58-an-easy-way-to-learn-and-use-tensorflow)


Adicionalmente, para esta actividad vamos a necesitar la librería `numpy`.

In [1]:
import numpy as np
import tensorflow as tf

# %matplotlib inline   This line works only in colab

print(tf.version)
# Con el fin de poder comparar los resultados
np.random.seed(2)

<module 'tensorflow_core._api.v2.version' from 'c:\\_devtools\\python\\anaconda3\\envs\\venvgpujupyter\\lib\\site-packages\\tensorflow_core\\_api\\v2\\version\\__init__.py'>



### 1. 2 Uso de constantes, variables, sesiones y placeholders tensorflow 2.0

TensorFlow 1.X requires users to manually stitch together an abstract syntax tree (the graph) by making tf.* API calls. It then requires users to manually compile the abstract syntax tree by passing a set of output tensors and input tensors to a session.run() call. TensorFlow 2.0 executes eagerly (like Python normally does) and in 2.0, graphs and sessions should feel like implementation details(within a tf.function, code with side effects execute in the order written).

TensorFlow 1.X relied heavily on implicitly global namespaces.You could then recover that tf.Variable, but only if you knew the name that it had been created with.Variable scopes, global collections, helper methods like tf.get_global_step(), tf.global_variables_initializer() were used to make the developer know the name of the variable.

!!!TensorFlow 2.0 eliminates all of these mechanisms in favor of the default mechanism: Keep track of your variables! If you lose track of a tf.Variable, it gets garbage collected.

The requirement to track variables creates some extra work for the user, but with Keras objects (see below), the burden is minimized.

#### Functions, not sessions

#### TensorFlow 1.X
outputs = session.run(f(placeholder), feed_dict={placeholder: input})
#### TensorFlow 2.0
outputs = f(input)

### 1. 3 Uso de constantes, variables, sesiones y placeholders

Ahora que hemos importado las librerías, comencemos con un ejercicio sencillo para implementar en Tensorflow. Escribamos el código que permita realizar el cálculo del error en la predicción de la salida $\hat{Y}$ para un ejemplo de entrenamiento con respecto a la salida esperada $Y$. Esto es:

$loss = \mathcal{L}(\hat{y}, y) = (\hat y^{(i)} - y^{(i)})^2$

Para este ejercicio:
- Definamos `y_hat = 25` y `y=29` como constantes. Podemos definir constantes en tensorflow de la siguiente manera `nom_constante = tf.constant(valor, dtype)`. Como se explico en TF 2.X ya no es importante el nombre de la variable pero olvidemos el typo de dato.

- Definamos la variable `loss`que debe almacenar el resultado del cálculo. Podemos definir variables en tensorflow de la siguiente manera `nom_variable = tf.Variable(valor, dtype)`

- Inicialicemos las variables. No es requerido.

- Ejecutemos la inicialización y el cálculo de `loss`. Se ejecutara automaticamente cuando se use.

Para profundizar en el tema de uso de constantes, variables y sesiones se recomienda consultar las siguientes los siguiente enlaces:

- [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)
- [tf.constant](https://www.tensorflow.org/api_docs/python/tf/constant)
- [Tensorflow 2.0](https://www.tensorflow.org/guide/effective_tf2)

Complete el código faltante en la siguiente celda, ejecute y verifique que el resultado concuerda con la salida esperada.

In [2]:
## Scalars
string = tf.Variable("this is a string", tf.string)
number = tf.Variable(324, tf.int16)
floatVar = tf.Variable(4.99, tf.float64)

## Vectors
vectorStr = tf.Variable(["one","two","three"], tf.string)
vectorInt = tf.Variable([1,2,3], tf.int16)

## Matrix
vectorInt2 = tf.Variable([[1,2,3], [4,5,6]], tf.int16)

# Lets check the dimentions
print(tf.rank(string))
print(tf.rank(vectorStr))
print(tf.rank(vectorInt2))

#shapes/reshape
print(vectorInt2.shape)
print(vectorInt2)
reshapeVec = tf.reshape(vectorInt2, [3,2])
reshapeVec = tf.reshape(vectorInt2, [3,-1])  # with -1 TF calculates the value
print(reshapeVec)


tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
(2, 3)
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>
tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32)


In [3]:
y_hat = tf.constant(29, name = 'y_hat')                    #Definir una constante y_hat = 25
y = tf.constant(25, name = 'y')                            #Definir una constante y = 29
loss = tf.Variable((y-y_hat)**2, name='loss')              #Definir una variable y asignar (y-y_hat)**2
print("loss =")
tf.print(loss)                                             #Ejecutar el cálculo de loss al imprimir

loss =
16


$\textbf{Salida esperada: }$

`loss = 16`

Recordemos que escribir y ejecutar programas en TensorFlow requiere seguir los siguientes pasos:

1. Crear Tensors (por ejemplo, constantes y variables) que serán ejecutas/evaluadas posteriormente.
2. Definir operaciones entre los Tensores.
5. Ejecutar

Por eso, cuando creamos la variable para el error, simplemente se define $loss$ en función de otros elementos, pero no se evalua su valor. Para evaluarla, primero tenemos que ejecutar.

Veamos otro ejemplo sencillo. En este caso, escribamos un programa con TensorFlow que realice la multiplicación de dos constantes. `a = 2` y `b = 10`. Almacene el resultado en una variable llamada `c` y al final imprima el resultado.

Tip:
- Para realizar la multiplicación en TensorFlow, podemos utilizar `tf.multiply(   ,   )`.

In [4]:
a = tf.constant(2, name = 'a')
b = tf.constant(10, name = 'b')
c =  tf.multiply(a,b) 
print(f"c = {c}")


c = 20


$\textbf{Salida esperada:}$

`c = Tensor("Mul:0", shape=(), dtype=int32)`

`c = 20`

Observe que la primer instrucción `print` no imprime el valor de la variable `c` (aun no ha sido evaluada). En resumen, recuerde inicializar sus variables, crear su sesión y ejecutar las operaciones dentro de la sesión.

### Los placeholders no existen en TF 2.X.

In [5]:
z = tf.constant(3, name = 'z')
r = 3*(z)
print(f"r = {r}")


r = 9


En el ejemplo anterior, cuando definimos `z` no se tiene que especificar su valor. Un `placeholder` es simplemente una variable a la que asignaras un valor posteriormente, cuando se ejecute `Session`. Normalmente se alimentan los datos al `placeholder` mediante un `feed_dictionary` en el momento que se ejecuta `Session`.

En resumen, el uso de `placeholders` permite a TensorFlow especificar las operaciones necesarias para un cálculo, es decir, construir un gráfo de cálculo pero en donde los valores se especificarán más adelante.

## 2.  Algunos problemas para calentar motores

### 2.1. Implementación de una función lineal

Implemente una función (método si prefiere POO) que utilice TensorFlow para calcular la siguiente ecuación: $Y = WX + b$, donde $W$ y $X$ son matrices aleatorias y $b$ es un vector aleatorio.

Para este ejercicio, considere las siguientes dimensiones: $W$:(4, 3), $X$: (3, 1), y $b$:(4, 1). A manera de ejemplo, observe la definición de la constante $X$ que tiene la forma (3, 1):
```python
X = tf.constant(np.random.randn(3,1), name = "X")
```

$\textbf{Nota}$: pueden resultar útil el uso de las siguientes funciones:
- `tf.mul(   ,   )` para realizar la multiplicación de dos matrices
- `tf.add(   ,   )` para realizar la suma
- `tf.random.randn(  )` para inicializar de manera aleatoria

In [6]:
def linear_function():
    np.random.seed(1)
    
    #Inicialice las constantes X, W, b
    X = tf.constant(np.random.randn(3,1), name = "X")
    W = tf.constant(np.random.randn(4,3), name = "W")
    b = tf.constant(np.random.randn(4,1), name = "b")

    #Defina las operaciones del grafo de cómputo
    Y = tf.add(tf.matmul(W,X), b)
    
    #Crear la sesión y ejecutarla
    
    return Y

print(f"result = {linear_function()}")

result = [[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]]


$\textbf{Salida esperada: }$

`result = [[-2.15657382]`

` [ 2.95891446]`

` [-1.08926781]`

` [-0.84538042]]`

### 2.2. Cálculo de la sigmoidal

TensorFlow provee un variedad de funciones comúnmente utilizadas para redes neuronales como:
- `tf.sigmoid()`
- `tf.softmax()`

Para este ejercicio, implemente una función para calcular la función sigmoidal de una entrada. Para implementar el ejercico, utilicemos el `placeholder` $x$. Cuando ejecutemos la sesión, utilicemos un `feed dictionary` para asignar el valor de entrada $z$. En resumen, tenemos que seguir los siguientes pasos:

- Crear un `placeholder` llamado x
- Definir las operaciones necesarias para calcular la función sigmoidal
- Ejecutar la sesión.

In [7]:
def sigmoid(z):
    # Crear tensor
    X = tf.constant(z, tf.float64)
    
    # Definir el cálculo de la sigmoidal
    result = tf.sigmoid(X)
    return result

print(f"sigmoid(0) = {sigmoid(0)}")
print(f"sigmoid(12) = {sigmoid(12)}")

sigmoid(0) = 0.5
sigmoid(12) = 0.9999938558253978


$\textbf{Salida esperada:}$

`sigmoid(0) = 0.5`

`sigmoid(12) = 0.9999938011169434`

## 2.3. Calcular la función de costo

Anteriormente implementamos la función de costo desde cero: 

$J = - \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log \sigma(z^{[2](i)}) + (1-y^{(i)})\log (1-\sigma(z^{[2](i)})\large )\small$

sin embargo, en TensorFlow no es necesario.

En este ejercicio, implementemos una función que calcule el costo, para esto utilizaremos la función: `tf.nn.sigmoid_cross_entropy_with_logits(logits = ...,  labels = ...)`

Tu implementación debe recibir `logits` (nuestros valores `z`), calcular la sigmoidal para obtener la activación y entonces calcular la función de costo.

In [8]:
def cost(logits, labels):
    """
    Parámetros:
    
    logits  vector que contiene las entradas a la unidad de activación (antes de la activación sigmoidal final)
    labels  vector de etiquetas (1 o 0)
    """
    
    # Crear tensors
    z = tf.constant(logits, tf.float64)
    y = tf.constant(labels, tf.float64)
    
    # Utilice la función de costo
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits = z,  labels = y)
    
    
    return cost

logits = sigmoid(np.array([0.2,0.4,0.7,0.9]))
cost = cost(logits, np.array([0,0,1,1]))
print (f"cost = {cost}")

cost = [1.00538722 1.03664083 0.41385432 0.39956614]


$\textbf{Salida esperada:}$

`cost = [1.0053872  1.0366409  0.41385433 0.39956614]`

## 2.4. Representación One Hot

Muchas veces en aprendizaje profundo tendremos un vector con números que van desde 0 a $C$-1, donde $C$ es el número de clases. Si $C$ es 4, por ejemplo, entonces podría tener el siguiente vector y que deberá convertir de la siguiente manera:

<img src="images/one_hot.png" style="width:600px;height:150px;">

A esto se le llama representación `One Hot`, porque en la nueva representación únicamente un elemento por columna se encuentra encendido (valor a 1). La implementación de esta transformación puede requerir alguna lineas de código en `numpy`, sin embargo, en TensorFlow podemos utilizar la siguiente función:

- tf.one_hot(labels, depth, axis) 

Implementemos una función que reciba un vector de etiquetas y el número de clases. La función debe retornar una representación "One Hot".

In [9]:
def one_hot_encoding(label, C):
    C = tf.constant(C, tf.int32)
    
    one_hot_encode = tf.one_hot(labels, C, axis=0)
       
    return one_hot_encode

labels = np.array([1,2,0,2,1,0])
one_hot = one_hot_encoding(labels, C = 3)
print (f"{one_hot}")

[[0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]


$\textbf{Salida esperada:}$

`[[0. 0. 1. 0. 0. 1.]`

` [1. 0. 0. 0. 1. 0.]`

` [0. 1. 0. 1. 0. 0.]]`

## 2.5. Inicializar un vector con ceros y unos

Para inicializar un vector con ceros y unos. La función que utilizaremos es `tf.ones ()`. Para inicializar con ceros, puede usar `tf.zeros ()` en su lugar. Estas funciones toma un `shape` y retorna una matriz con la misma dimensiones pero llena de ceros y unos, respectivamente.

Implementemos la siguiente función para basados en un `shape`, devolver una matriz (de la misma dimensión con unos).

In [10]:
def ones(shape):
    
    ones = tf.ones(shape)
    
    return ones

print (f"{ones([3])}")

[1. 1. 1.]


$\textbf{Salida esperada:}$

`[1. 1. 1.]`